# Extracció keywords

In [7]:
import emoji
from textblob import TextBlob
import pandas as pd
import numpy as np
import regex
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\theda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Funcions per treballar amb keywords

In [8]:
import emoji
#Funcio per treure els emojis d'un text.
def deEmojify(text):

    string = ""
    data = regex.findall(r'\X', text)
    for word in data:
        for char in word:
            if not char in emoji.UNICODE_EMOJI:
                string = string + char

    return string

def removeStopwords(texto):
    texto = texto.lower()
    texto = re.sub('_', '', texto)
    texto = re.sub('@', '', texto)
    texto = re.sub(':', '', texto)
    texto = re.sub('/', '', texto)
    texto = re.sub('!', '', texto)
    texto = re.sub('_', '', texto)
    texto = re.sub('\.', '', texto)
    texto = deEmojify(texto)
    blob = TextBlob(texto).words
    outputlist = [word for word in blob if word not in stopwords.words('spanish')]
    return(outputlist)

def normalizeString(l):
    l = l.fillna('').values.tolist()
    for user in l:
        user[0] = removeStopwords(user[0])
        user[1] = removeStopwords(user[1]) 
    return l

def wordDict(l):
    dt = {}
    for sublist in l:
        for item in sublist:
            for item2 in item:
                if item2 not in dt:
                    dt[item2] = 1
                else:
                    dt[item2] += 1
    
    for key in list(dt.keys()):
        if len(key) <= 1:  
            del dt[key] 
    
    return dt

def countWords(lt):
    mainDict = {}
    for emoji in lt:
        if emoji in mainDict:
            mainDict[emoji] += 1
        else:
            mainDict[emoji] = 1
    return mainDict

## Extracció keywords

In [ ]:
import emoji

pp = pd.read_csv("pp.csv")
psoe = pd.read_csv("psoe.csv")
cs = pd.read_csv("cs.csv")
vox = pd.read_csv("vox.csv")
up = pd.read_csv("up.csv")

#agafem de cada dataframe de cada partit els noms dels usuaris i les descripccions 
voxUD = vox[["name","description"]]
ppUD = pp[["name","description"]]
csUD = cs[["name","description"]]
psoeUD = psoe[["name","description"]]
upUD = up[["name","description"]]

#per tot el dataset, treiem les stopwords i natejem tots els caracters que no son utils
voxWords = normalizeString(voxUD)
#fem un diccionari amb totes les paraules recollides
voxWordsFlat =  wordDict(voxWords)        
#ordenem el diccionari per ocurrencies i eliminem les que tinguin menys de 3
voxWordsFlat = {k: v for k, v in sorted(voxWordsFlat.items(), reverse=True, key=lambda item: item[1])}
voxWordsFlat = {k: v for k, v in voxWordsFlat.items() if v > 2}
#fem un llistat de tuples amb la paraula i la quantitat de cops que surt
voxWordsFlat = list(voxWordsFlat.items())

#repetim el proces per els altres partits

ppWords = normalizeString(ppUD)
ppWordsFlat =  wordDict(ppWords)        
ppWordsFlat = {k: v for k, v in sorted(ppWordsFlat.items(), reverse=True, key=lambda item: item[1])}
ppWordsFlat = {k: v for k, v in ppWordsFlat.items() if v > 2}
ppWordsFlat = list(ppWordsFlat.items())

csWords = normalizeString(csUD)
csWordsFlat =  wordDict(csWords)        
csWordsFlat = {k: v for k, v in sorted(csWordsFlat.items(), reverse=True, key=lambda item: item[1])}
csWordsFlat = {k: v for k, v in csWordsFlat.items() if v > 2}
csWordsFlat = list(csWordsFlat.items())

psoeWords = normalizeString(psoeUD)
psoeWordsFlat =  wordDict(psoeWords)        
psoeWordsFlat = {k: v for k, v in sorted(psoeWordsFlat.items(), reverse=True, key=lambda item: item[1])}
psoeWordsFlat = {k: v for k, v in psoeWordsFlat.items() if v > 2}
psoeWordsFlat = list(psoeWordsFlat.items())

upWords = normalizeString(upUD)
upWordsFlat =  wordDict(upWords)        
upWordsFlat = {k: v for k, v in sorted(upWordsFlat.items(), reverse=True, key=lambda item: item[1])}
upWordsFlat = {k: v for k, v in upWordsFlat.items() if v > 2}
upWordsFlat = list(upWordsFlat.items())


#llista amb les keywords y ocurrencies per a cada partit. 
fList = [voxWordsFlat,ppWordsFlat,csWordsFlat,psoeWordsFlat,upWordsFlat]

flatTrans = []
for sublist in fList:
    for item in sublist:
        flatTrans.append(item[0])

#Conjunt de les keywords diferents que apareixen (tenint en compte tots els partits politics). Util per quan calculem el tfidf.
diffWords = set(flatTrans)

## Proves per a calcular l'score d'un tweet

In [ ]:
DannyTw = "soy de vox"

DannyTwClean = removeStopwords(DannyTw)
DannyDict = countWords(DannyTwClean)
DannyList = list(DannyDict.items())



wordIdf = {}
for word in diffWords:
    wordIdf[word] = 0
    for index, party in enumerate(fList):
        for partyWord, num in party:
            if partyWord == word:
                wordIdf[word] += 1


voxWordNum = np.sum([value for i, value in fList[0]])
ppWordNum = np.sum([value for i, value in fList[1]])
csWordNum = np.sum([value for i, value in fList[2]])
psoeWordNum = np.sum([value for i, value in fList[3]])
upWordNum = np.sum([value for i, value in fList[4]])
WordPartiesSum = [voxWordNum, ppWordNum, csWordNum, psoeWordNum, upWordNum]

scores = {}
for index, party in enumerate(fList):
    scores[index] = 0
    
print(DannyList)

for i, value in DannyList:
    for index, party in enumerate(fList):
        for word, num in party:
            if i == word:
                scores[index] += value * np.log(len(fList)/wordIdf[word]) * num/WordPartiesSum[index]

print(scores)
list(scores.values())/np.sum(list(scores.values()))